### Libraries

In [1]:
######## Libraries #######################################
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from itertools import permutations
from collections import deque
from typing import Union

### Functions

In [2]:
######## Functions #######################################
def toStr(vec) -> str:
    return ''.join(map(str, vec))

def toInt(vec) -> int:
    return int("".join(str(i) for i in vec), 2)

def toBinVec(x, l=0) -> list:                
    result = [int(i) for i in bin(x)[2:]]
    missing_zeros = l-len(result)
    if (missing_zeros > 0):
        for i in range(missing_zeros):
            result.insert(0, 0)
    return result

def hammingweight(vec) -> int:
    return np.count_nonzero(vec == 1)

def rotate_left_vec(vec) -> list:
    shifted = vec.copy()
    temp = deque(shifted)
    temp.rotate(-1)
    return list(temp)

def calculate_c(n: int, U: int) -> int:
    return 2**n-3**U

def calculate_z(parity_vector: []) -> int:
    indices = []
    for i, entry in enumerate(parity_vector, start=0):
        if entry == 1:
            indices.append(i)
    result = 0
    U = len(indices)
    for i, entry in enumerate(indices, start=0):
        result = result + 3**(U-(i+1)) * 2**(entry)
    return result

def getCycle(start, l):
    cycle = np.array(calculate_z(start))
    pos = start
    for i in range(l-1):
        pos = rotate_left_vec(pos)
        cycle = np.hstack([cycle, calculate_z(pos)])
    return cycle

########## Search Functions ################################
def find_extrema(vec) -> Union[int,int,int,list,list]:
    L = len(vec)
    vecToInt = toInt(vec)
    min = vecToInt
    max = 0
    odd_max = 0
    vec_min = vec
    vec_max = vec
    shifted = vec.copy()
    for i in range(L):
        if vecToInt < min:
            min = vecToInt
            vec_min = shifted
        if vecToInt > max:
            max = vecToInt
            vec_max = shifted
        temp = deque(shifted)
        temp.rotate(-1)
        shifted = list(temp)
        vecToInt = toInt(shifted)
    return min, max, odd_max, vec_min, vec_max

def find_rotational_distance(vec1, vec2) -> Union[int]:
    L = len(vec1)
    vec1ToInt = toInt(vec1)
    vec2ToInt = toInt(vec2)
    left_shifts = 0
    shifted = vec1.copy()
    for i in range(L):
        if toInt(shifted) == vec2ToInt:
            left_shifts = i
            break
        temp = deque(shifted)
        temp.rotate(-1)
        shifted = list(temp)
    return left_shifts

######### Prepare Data ##############################
def setInput(l) -> list:                
    delay = np.zeros((l,), dtype=int)
    for i in range(2**l-1):
        delay = np.vstack([delay, toBinVec(i+1, l)])
    INPUT = delay
    return INPUT

def getParam(n, t):
    binominal = math.comb(n, t)
    gcd = math.gcd(n, t)
    if (gcd == 1) or (t == 0) or (t == n):
        case = 'Trivial'
    else:
        case = 'Nontrivial'
    cases = binominal/n
    return binominal, gcd, case, cases

def getData(l) -> list:
    B = setInput(l)
    INPUT = B[0]      ## Zeilen auslesen
    DATA = setData(INPUT, l)
    for i in range(2**l-1):
        remain = B[i+1]      ## Zeilen auslesen
        DATA = np.vstack([DATA, setData(remain, l)])
        INPUT = np.vstack([INPUT, remain])
    return DATA, INPUT

### Prepare Dataframe

In [3]:
def setData(B_vec, l) -> list:
    EXP = find_extrema(B_vec)                                   # Extrema
    l = len(B_vec)                                              # Länge
    U = hammingweight(B_vec)                                    # Hamming weight    
    cycle = getCycle(B_vec, l)                                  # z(B) Cycle
    param = getParam(l, U)                                      # BinomKoeff & Cases
    diff = (U*find_rotational_distance(EXP[3], EXP[4])+1)/l     # Division
    Z = sum(cycle)                                              # Z(B)
    c = calculate_c(l, U)                                       # c
    
    ######## U # B_min # min ## Binom ### gcd ##### case ### diff # c # cycle # Z(B)
    DATA = [[U, EXP[3], EXP[0], param[3], param[1], param[2], diff, c, cycle, Z]]
    
    return DATA

def getDataframe(DATA, n):
    df_allData = pd.DataFrame(DATA, columns = ['U', 'B_min', 'min', 'Binom', 'gcd', 'case', 'Diff', 'c', 'cycle', 'Z(B)'])
    pd.set_option('display.expand_frame_repr', True)
    df_copy = df_allData.copy()
    df_roots = df_copy.drop_duplicates(subset="min")
    df_roots = df_roots.drop(columns=['min'])
    df_roots = df_roots.rename(columns={'B_min': 'Roots'})
    df_roots = df_roots.sort_values(by=['U'])


    return df_allData, df_roots

def getCardinalityDataframe(df, t):
    df_copy2 = df.copy()
    if (t == 'all'):
        df_cardinalitys = df_copy2
    else:
        df_cardinalitys = df_copy2[(df_copy2.U == t)]
    df_cardinalitys = df_cardinalitys.sort_values(by=['min'])
    df_cardinalitys = df_cardinalitys.groupby(['U', 'min'])[['min']].count()

    return df_cardinalitys

def getConstantCardinalityDataframe(df, t):
    df_copy2 = df.copy()
    if (t == 'all'):
        df_constantcardinalitys = df_copy2
    else:
        df_constantcardinalitys = df_copy2[(df_copy2.U == t)]
    df_constantcardinalitys = df_constantcardinalitys.sort_values(by=['Z(B)'])
    df_constantcardinalitys = df_constantcardinalitys.groupby(['U', 'Z(B)'])[['Z(B)']].count()

    return df_constantcardinalitys

### Prepare Equivalence Class

In [4]:
def RotaitionMatrix(n):
    A = np.zeros((n, n), int)
    np.fill_diagonal(A, 1)
    for i in range(n-1):
        A[n-1-i,:] = A[n-1-1-i,:]
    A[0,:] = np.zeros((1,n), int)
    A[0,n-1] = 1
    return A

def getVector(vector, A):
    n = len(vector)
    U = np.zeros((n, n), int)
    U[0, :] = vector
    for i in range(n-1):
        U[i+1, :] = np.matmul(U[i, :], A)
    return U

def setEquivialnedClassData(Matrix, k) -> list:
    B_vec = Matrix[0][k]                          
    B_int = toInt(B_vec)                            
    cycle = Matrix[1][k]                          
    Z = sum(cycle)
    
    DATA = [[B_vec, B_int, cycle, Z]]
    return DATA

def getEquivialnedClassData(Dataframe, index, A):
    vector = Dataframe.iloc[index]['Roots']
    cycle = Dataframe.iloc[index]['cycle']
    Matrix = [getVector(vector, A), getVector(cycle, A)]
    for i in range(n):
        if (i==0):
            EquiClassDATA = setEquivialnedClassData(Matrix, i)
        else:
            EquiClassDATA = np.vstack([EquiClassDATA, setEquivialnedClassData(Matrix, i)])
    return EquiClassDATA

def getEquivialnedClassDataframe(DATA, n):
    df_equiclass = pd.DataFrame(EquiClassDATA, columns = ['B_vec', 'B_int', 'Cycle', 'Z(B)'])
    pd.set_option('display.expand_frame_repr', True)
    return df_equiclass

def getCardinals(df):
    for i in list(df):
    # show the list of values  
        D = df[i].tolist()
    D = list(dict.fromkeys(D))
    if len(D) == 1:
        case = 'trivial'
    else:
        case = 'nontrivial'
    return D, case

### Get Dataframe

In [5]:
############ MAIN ###########################
### Inputs ####
n = 6
### Operations ####
[DATA, INPUT] = getData(n)
Dataframe = getDataframe(DATA, n)

### Show Dataframe ####
Dataframe[1]
# df_allData       ->0
# df_roots         ->1

C:\Users\adrian.AUENLAND\Anaconda3\lib\site-packages\numpy\core\_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


,U,Roots,Binom,gcd,case,Diff,c,cycle,Z(B)
0,0,"[0, 0, 0, 0, 0, 0]",0.166667,6,Trivial,0.166667,63,"[0, 0, 0, 0, 0, 0]",0
1,1,"[0, 0, 0, 0, 0, 1]",1,1,Trivial,1,61,"[32, 16, 8, 4, 2, 1]",63
3,2,"[0, 0, 0, 0, 1, 1]",2.5,2,Nontrivial,1.5,55,"[80, 40, 20, 10, 5, 35]",190
5,2,"[0, 0, 0, 1, 0, 1]",2.5,2,Nontrivial,1.16667,55,"[56, 28, 14, 7, 38, 19]",162
9,2,"[0, 0, 1, 0, 0, 1]",2.5,2,Nontrivial,0.833333,55,"[44, 22, 11, 44, 22, 11]",154
7,3,"[0, 0, 0, 1, 1, 1]",3.33333,3,Nontrivial,1.66667,37,"[152, 76, 38, 19, 47, 89]",421
11,3,"[0, 0, 1, 0, 1, 1]",3.33333,3,Nontrivial,2.16667,37,"[116, 58, 29, 62, 31, 65]",361
13,3,"[0, 0, 1, 1, 0, 1]",3.33333,3,Nontrivial,1.16667,37,"[92, 46, 23, 53, 98, 49]",361
21,3,"[0, 1, 0, 1, 0, 1]",3.33333,3,Nontrivial,0.666667,37,"[74, 37, 74, 37, 74, 37]",333
15,4,"[0, 0, 1, 1, 1, 1]",2.5,2,Nontrivial,1.5,-17,"[260, 130, 65, 89, 125, 179]",848


#### Show Equivalence Class

In [6]:
index = 6

EquiClassDATA = getEquivialnedClassData(Dataframe[1], index, RotaitionMatrix(n))
getEquivialnedClassDataframe(EquiClassDATA, n)
# for index in range(n):
#     EquiClassDATA = getEquivialnedClassData(Dataframe[2], index, RotaitionMatrix(n))
#     print(getEquivialnedClassDataframe(EquiClassDATA, n))

C:\Users\adrian.AUENLAND\Anaconda3\lib\site-packages\numpy\core\_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


,B_vec,B_int,Cycle,Z(B)
0,"[0, 0, 0, 1, 1, 0, 0, 1]",25,"[248, 124, 62, 31, 161, 356, 178, 89]",1249
1,"[0, 0, 1, 1, 0, 0, 1, 0]",50,"[124, 62, 31, 161, 356, 178, 89, 248]",1249
2,"[0, 1, 1, 0, 0, 1, 0, 0]",100,"[62, 31, 161, 356, 178, 89, 248, 124]",1249
3,"[1, 1, 0, 0, 1, 0, 0, 0]",200,"[31, 161, 356, 178, 89, 248, 124, 62]",1249
4,"[1, 0, 0, 1, 0, 0, 0, 1]",145,"[161, 356, 178, 89, 248, 124, 62, 31]",1249
5,"[0, 0, 1, 0, 0, 0, 1, 1]",35,"[356, 178, 89, 248, 124, 62, 31, 161]",1249
6,"[0, 1, 0, 0, 0, 1, 1, 0]",70,"[178, 89, 248, 124, 62, 31, 161, 356]",1249
7,"[1, 0, 0, 0, 1, 1, 0, 0]",140,"[89, 248, 124, 62, 31, 161, 356, 178]",1249


## Analize Equivalence Class
#### Prepare Cardinality Dataframe

In [10]:
### Prepare Cardinality Dataframe ###
CardinalityDataframe = [getCardinalityDataframe(Dataframe[0], 'all'), getConstantCardinalityDataframe(Dataframe[0], 'all')]

### Show Cardinality Dataframe ###
# CardinalityDataframe[0]

# df_cardinality   ->0
# df_cardilconstant->1

#### Show Cardinalities for all Hammingweights

In [15]:
### Get Cardinalities ####
print('Cardinalities D(i):')
for U in range(n+1):
    cardinalityDataframe = getCardinalityDataframe(Dataframe[0], U)
    cardinalities = getCardinals(cardinalityDataframe)
    print(U, cardinalities)
### Get Constant cardinalitys ####
print('\nConstantCardinalities Z(i):')
for U in range(n+1):
    constantCardinalityDataframe = getConstantCardinalityDataframe(Dataframe[0], U)
    constantcardinalities = getCardinals(constantCardinalityDataframe)
    print(U, constantcardinalities)

Cardinalities D(i):
0 ([1], 'trivial')
1 ([8], 'trivial')
2 ([8, 4], 'nontrivial')
3 ([8], 'trivial')
4 ([8, 4, 2], 'nontrivial')
5 ([8], 'trivial')
6 ([8, 4], 'nontrivial')
7 ([8], 'trivial')
8 ([1], 'trivial')

ConstantCardinalities Z(i):
0 ([1], 'trivial')
1 ([8], 'trivial')
2 ([4, 8], 'nontrivial')
3 ([8, 16], 'nontrivial')
4 ([2, 16, 8, 4], 'nontrivial')
5 ([8, 16], 'nontrivial')
6 ([4, 8], 'nontrivial')
7 ([8], 'trivial')
8 ([1], 'trivial')


#### Number of Elements per Equivalence Class C(n,t,d)

In [16]:
#### How many classes per Hammingweight and given length n ##########
n = n
t = 4   ##Hammingweight t = U = hw = N(B)
k = 0
array = getCardinals(getCardinalityDataframe(Dataframe[0], t))
D = np.sort(array[0])
print('Cardinalitys: D =',D,'\n')
print('C( n , t , d ) = 1 / d*(( d // t * d / n) - k*C(n, t, k))','\n')
C = np.zeros((len(D)+1), int)

for i in range(len(D)):
    summe = 0
    d = D[i]
    for j in range(d):
        if (j in D):
            if (j < d):
                k = j
            summe = summe + k*C[i]
    c = (1/d)*(math.comb(d, (int)(t*d/n)) - summe)
    C[i+1] = c
    print('C(',n,',',t,',',d,' )=','1 /',d,'((',d,'//',t,'*',d,'/',n,')-',summe,') = ',c,' cases')

Cardinalitys: D = [2 4 8] 

C( n , t , d ) = 1 / d*(( d // t * d / n) - k*C(n, t, k)) 

C( 8 , 4 , 2  )= 1 / 2 (( 2 // 4 * 2 / 8 )- 0 ) =  1.0  cases
C( 8 , 4 , 4  )= 1 / 4 (( 4 // 4 * 4 / 8 )- 2 ) =  1.0  cases
C( 8 , 4 , 8  )= 1 / 8 (( 8 // 4 * 8 / 8 )- 6 ) =  8.0  cases


In [80]:
binominal = math.comb(10, 5)
gcd = math.gcd(10, 5)

check = binominal/10

print(binominal, gcd, check)
t = 4056*60
print(t)
print(t+115)
print(t+5)

print(1/8*(math.comb(8, 4)-2-4))

252 5 25.2
243360
243475
243365
8.0
